# Pair Programming 
## Lección 15 - ETL III - Transformación II - Clases y funciones de limpieza

En la lección de hoy aprendimos como Crearnos una clase que nos permita limpiar los datos obtenidos de la API.

En este ejercicio, tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

Happy coding 🦈

In [2]:
import pandas as pd
pd.options.display.max_columns=None
import requests
import ast
import numpy as np

In [ ]:
class Etl:
    # constructor
    def __init__(self, latitud, longitud, producto):
        
        self.latitud = latitud
        self.longitud = longitud
        self.producto = producto

    # Método para llamar a la API
    def llamar_api(self, producto, diccionario):
        df_meteo= pd.DataFrame()

        for k,v in diccionario.items():
            df_meteo1 = pd.DataFrame()
            # Los values de latitud y longitud serían = latitud: v[0] y longitud: v[1]. 
            producto = "meteo"
            response = requests.get(url = f'http://www.7timer.info/bin/api.pl?lon=-{v[1]}&lat={v[0]}&product={self.producto}&output=json') 
            df_meteo1 = pd.DataFrame.from_dict(pd.json_normalize(response.json()["dataseries"]))
            df_meteo1["pais"] = k

            df_meteo = pd.concat([df_meteo1, df_meteo], axis=0, ignore_index=True) 
            
            if response.status_code == 200:
                print('La peticion se ha realizado correctamente, se ha devuelto el código de estado:',response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 402:
                print('No se ha podido autorizar usario, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            elif response.status_code == 404:
                print('Algo ha salido mal, el recurso no se ha encontrado,se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason)
            else:
                print('Algo inesperado ha ocurrido, se ha devuelto el código de estado:', response.status_code,' y como razón del código de estado: ',response.reason) 
        return df_meteo   

    def limpiar_rh_profile(self, df_meteo):
        # Función que limpia la columna rh_profile del dataframe

        # Resetear el índice del dataframe para no tener columnas repetidas
        df_meteo.reset_index(inplace=True)

        # Utilizamos la librería ast para castear un string que dentro tiene un diccionario
        df_meteo['rh_profile']= df_meteo['rh_profile'].apply(ast.literal_eval)
        # Convertimos a serie la columna que en su interior tiene un diccionario
        df_rh = df_meteo["rh_profile"].apply(pd.Series)

        # Creamos un dataframe vacío para insertar cada uno de los values del diccionario
        df_rh_profile = pd.DataFrame()
        for i in range(len(df_rh.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "rh_" + str(df_rh[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_rh[i].apply(pd.Series)["rh"] )
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_rh_profile.insert(i, nombre, valores)
        return df_rh_profile

    def limpiar_wind_profile(self, df_meteo):
        # Función que limpia la columna wind_profile del dataframe

        # Resetear el índice del dataframe para no tener columnas repetidas
        df_meteo.reset_index(inplace=True)
         # Utilizamos la librería ast para castear un string que dentro tiene un diccionario
        df_meteo['wind_profile']= df_meteo['wind_profile'].apply(ast.literal_eval)
        # Convertimos a serie la columna que en su interior tiene un diccionario
        df_wind = df_meteo["wind_profile"].apply(pd.Series)

        # Primero cambiamos el value de direction
        df_wind_direction_profile = pd.DataFrame()
        for i in range(len(df_wind.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_direction_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_wind[i].apply(pd.Series)["direction"])
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_wind_direction_profile.insert(i, nombre, valores)

        # Ahora cambiamos los values de speed
        df_wind_speed_profile = pd.DataFrame()
        for i in range(len(df_wind.columns)): 
            #aplicamos el apply,extraemos el valore de la key "layer" y lo almacenamos en una variable que convertimos a string 
            nombre = "wind_speed_" + str(df_wind[i].apply(pd.Series)["layer"][0]) 
            #hacemos lo mismo con una variable que se llame valores para "guardar" los valores de la celda
            valores = list(df_wind[i].apply(pd.Series)["speed"])
            #usamos el método insert de los dataframes para ir añadiendo esta información a el dataframe con la información del clima. 
            df_wind_speed_profile.insert(i, nombre, valores)
        return df_wind_direction_profile, df_wind_speed_profile

    def concatenar_columnas_profile(self, df_rh_profile, df_wind_direction_profile, df_wind_speed_profile):
        df_profile = pd.concat([df_wind_direction_profile, df_wind_speed_profile, df_rh_profile], axis=1)
        return df_profile

    def eliminar_columnas_profile(self, df_meteo):
        df_meteo = df_meteo.drop(["rh_profile", "wind_profile"], axis=1)
        print(f"Ahora df_meteo tiene una formato de (filas,columnas): {df_meteo.shape}")
        return df_meteo

    def unir_df_meteo_profile(self, df_meteo, df_profile):
        df_meteo_completo = df_meteo.join(df_profile, how="inner", on="index")
        return df_meteo_completo

    def agrupar_paises_media(self, df_meteo_completo):
        df_grupo = df_meteo_completo.groupby("pais")[df_meteo_completo.columns].mean().reset_index()
        return df_grupo

    def unir_df_grupo_ataques(self, df_grupo, df_ataques):
        df_completo = df_ataques.merge(df_grupo, how="inner", right_on="pais", left_on="country")
        return df_completo